**train.csv**: A full training dataset with the following attributes:

**id**: unique id for a news article <br>
**title**: the title of a news article <br>
**author**: author of the news article <br>
**text**: the text of the article; could be incomplete <br>
**label**: a label that marks the article as potentially unreliable <br>
1: unreliable <br>
0: reliable

**test.csv**: A testing training dataset with all the same attributes at train.csv without the label.
**submit.csv**: A sample submission that you can

### 1. Importing Depencies

In [1]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

### 2. Removing 'Stopwords from data'

In [2]:
nltk.download('stopwords')
print(stopwords.words('English'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/arpitpatel/nltk_data...


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data]   Unzipping corpora/stopwords.zip.


### 3. Data Preprocessing

In [3]:
news_dataset=pd.read_csv('../Datasets/train.csv')

In [4]:
news_dataset.shape

(20800, 5)

In [5]:
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [6]:
news_dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [7]:
news_dataset=news_dataset.fillna('')

In [8]:
news_dataset['content']=news_dataset['author']+' '+news_dataset['title']

In [9]:
news_dataset.head()

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...


### 4. Stemming
- stemming is the process of reducing the words to its root word.
- **for example**:
- before stemming: actor,actress,acting
- after stemming: act

In [10]:
port_stem=PorterStemmer()

In [11]:
def stemming(content):
    stemmed_content=(re.sub('[^a-zA-Z]',' ',content)).lower().split()
    stemmed_content=[port_stem.stem(word) for word in stemmed_content if word not in stopwords.words('English')]
    stemmed_content=' '.join(stemmed_content)
    return stemmed_content

In [12]:
news_dataset['content']=news_dataset['content'].apply(stemming)

### 5. Dividing into features and target

In [13]:
features=news_dataset['content'].values
target=news_dataset['label'].values

### 6. Converting Textual Data to Numeric data

In [14]:
vectorizer=TfidfVectorizer()
vectorizer.fit(features)

TfidfVectorizer()

In [15]:
features=vectorizer.transform(features)

### 7. Train_Test_split

In [16]:
X_train,X_test,Y_train,Y_test=train_test_split(features,target,test_size=0.2,stratify=target,random_state=2)

### 8. Training the Model

In [17]:
model=LogisticRegression()

In [18]:
model.fit(X_train,Y_train)

LogisticRegression()

### 9. Model Evaluation

In [19]:
predicted_Y_train=model.predict(X_train)
accuracy_score(Y_train,predicted_Y_train)

0.9865985576923076

In [20]:
predicted_Y_test=model.predict(X_test)
accuracy_score(Y_test,predicted_Y_test)

0.9790865384615385

### 10. Creating Predictive System

In [21]:
input=X_test[0]
prediction=model.predict(input)
if(prediction==0):
    print('The News is Real.')
else:
    print('The News is Fake.')

The News is Fake.


In [22]:
Y_test[0]

1